In [ ]:
def init_filters(rs=None):
    if rs!=None:
        np.random.seed(rs)
    f1=np.random.normal(size=(5,5))
    f2=np.random.normal(size=(5,5))
    f3=np.random.normal(size=(5,5))
    f4=np.random.normal(size=(5,5))
    f5=np.random.normal(size=(5,5))
    f6=np.random.normal(size=(5,5))
    f7=np.random.normal(size=(5,5))
    f8=np.random.normal(size=(5,5))
    f9=np.random.normal(size=(5,5))
    f10=np.random.normal(size=(5,5))
    return f1,f2,f3,f4,f5,f6,f7,f8,f9,f10


def max_pool(arr,pool_filter=2):

    stride=pool_filter

    #applying single sided row padding when required 
    if arr.shape[0]%pool_filter==0:
        row_padding=0
    else:
        row_padding=pool_filter-(arr.shape[0]%pool_filter)
        
    #applying single sided col padding when required 
    if arr.shape[1]%pool_filter==0:
        col_padding=0
    else:
        col_padding=pool_filter-(arr.shape[1]%pool_filter)
    
    #creating padded matrix
    input=np.zeros((arr.shape[0]+row_padding,arr.shape[1]+col_padding))
    input[:arr.shape[0],:arr.shape[1]]=arr
    
    col_pooled=int((input.shape[1]-pool_filter)/stride)+1 #or int((arr.shape[0]+row_padding-pool_filter)/stride)+1
    row_pooled=int((input.shape[0]-pool_filter)/stride)+1 #or int((arr.shape[1]+col_padding-pool_filter)/stride)+1
    
    pooled=np.zeros((row_pooled,col_pooled))
    
    skeleton=np.zeros(input.shape)
    
    for i in range (0,row_pooled):
        for j in range(0,col_pooled):
            '''obtaining max_pool output'''
            pooled[i,j]=(input[(stride*i):(stride*i)+pool_filter,(stride*j):(stride*j)+pool_filter]).max()

            '''obtaining skeleton for max_pool back prop'''
            max=pooled[i,j]
            [p,q]=find_index(input[(stride*i):(stride*i)+pool_filter,(stride*j):(stride*j)+pool_filter],max)
            skeleton[(stride*i):(stride*i)+pool_filter,(stride*j):(stride*j)+pool_filter][p,q]=1
            # [(stride*i):(stride*i)+pool_filter,(stride*j):(stride*j)+pool_filter]  selects the current window the filter is over 
            # [p,q] is the index where max value is in the current max_pool filter window 
            
    return pooled,skeleton


def find_index(arr,v):
    for i in range(arr.shape[0]):
        for j in range(arr.shape[1]):
            if arr[i,j]==v:
                return [i,j]

def convolve(image,kernel,padding=1,stride=1):
    if padding != 0:
        arr=np.zeros((image.shape[0]+padding*2,image.shape[1]+padding*2))
        arr[padding:-1*padding,padding:-1*padding]=image
    else:
        arr=image
    
    col_out_dim=int((image.shape[1]+(2*padding)-kernel.shape[1])/stride)+1
    row_out_dim=int((image.shape[0]+(2*padding)-kernel.shape[0])/stride)+1

    output=np.zeros((row_out_dim,col_out_dim))

    for i in range (0,row_out_dim):
        for j in range(0,col_out_dim):
            output[i,j]=(kernel*arr[stride*i:((stride*i)+kernel.shape[0]),stride*j:(stride*j)+kernel.shape[1]]).sum()
    return output

def relu(input):
    return np.maximum(0,input)


def convolution_epoch(x,f):
    
    output=convolve(x,f)
    relu_op=relu(output)
    pooled,skeleton=max_pool(relu_op)
    flattened=pooled.reshape((169,1))
    return flattened,skeleton,output
    

def minmax_scaler(input):    
    min=input.min()
    max=input.max()
    scaled=(input - min)/(max-min)
    return scaled

def encoded(y):
  result=np.zeros((10,1))
  result[y,0]=1
  return result

def initialize_weights(rs=None):
    if rs!=None:
        np.random.seed(rs)
    w1=np.random.rand(10,1690) - 0.5
    b1=np.random.rand(10,1) - 0.5
    w2=np.random.rand(10,10) - 0.5
    b2=np.random.rand(10,1) - 0.5
    return w1,b1,w2,b2

def relu_der(z):
    return z>0

def softmax(z):
    return np.exp(z)/np.sum(np.exp(z))

def forward(x,w1,b1,w2,b2):
    z1=w1.dot(x) + b1
    a1=relu(z1)
    z2=w2.dot(a1) + b2
    a2=softmax(z2)
    return z1,a1,z2,a2

def back_prop(y_unencoded,scaled,z1,a1,z2,a2):
    
    dc_z2=a2-encoded(y_unencoded)
    dc_w2=dc_z2.dot((a1.T))
    dc_b2=dc_z2
    
    dc_z1=(w2.dot(dc_z2))*(relu_der(z1))
    dc_w1=dc_z1.dot((scaled.T))
    dc_b1=dc_z1

    dc_x=(w1.T).dot(dc_z1)
    
    return dc_w1,dc_b1,dc_w2,dc_b2,dc_x
def update_params(alpha,dc_w1,dc_b1,dc_w2,dc_b2,w1,b1,w2,b2):
    w1=w1-(alpha*dc_w1)
    b1=b1-(alpha*dc_b1)
    w2=w2-(alpha*dc_w2)
    b2=b2-(alpha*dc_b2)
    return w1,b1,w2,b2

def undo_maxpool(pooled,skeleton,filter_size=2):
    
    expanded_rows=pooled.shape[0]*filter_size
    expanded_cols=pooled.shape[1]*filter_size
    expanded=np.zeros((expanded_rows,expanded_cols))
    for i in range(len(pooled)):
        for j in range(len(pooled[i])):
            expanded[i*filter_size:i*filter_size+filter_size,j*filter_size:j*filter_size+filter_size]=(np.zeros((filter_size,filter_size))+1)*pooled[i,j]
    #print(expanded)
    return expanded*skeleton